In [1]:
import math
import random
from sympy import isprime, legendre_symbol, gcd
import numpy as np
import time

ModuleNotFoundError: No module named 'sympy'

# Post Quantum Cryptography


# I) What is RSA ?

## a) Math Introduction (base of the algorithm -> Factorisation of prime numbers)


---
#### First of all let's introduce some math aspects. The factorisation in prime numbers is one of the key concepts of RSA. 
#### What's a Prime number : A prime number is an integer greater than 1 that has only two divisors: 1 and itself. Examples: 2, 3, 5, 7, 11, 13,
#### Let's now speak about prime factorisation : Every integer can be written as a product of prime numbers and it's called prime factorization.
#### Some key examples : 28 = 2 × 2 × 7 = 2² × 7 or 91 = 7 × 13
#### This factorization is unique (apart from the order). This is the fundamental theorem of arithmetic.
---

## b) Presentation of RSA (History)

----
#### Before the 1970s, all cryptography relied on shared secret keys (symmetric encryption). But this raised a major problem: how could the secret key be exchanged securely if enemies were listening?
#### In response to it, in 1976, Whitfield Diffie and Martin Hellman published New Directions in Cryptography by introducing the idea of a public key to encrypt and a private key to decrypt so no need to secretly exchange keys in advance.
#### Building on this, researchers realized that multiplying two large prime numbers was a simple task, whereas factoring their product back into its prime components was extremely difficult.
#### This property, known as a “one-way function,” provided exactly the mathematical foundation needed for a secure encryption system. In 1977, Rivest, Shamir, and Adleman transformed this insight into a practical algorithm, which became known as RSA, named after their initials.
---

## c) Implementation of RSA (Algorithm)

The first step is the ***Key Generation*** : 
1) Choose two prime numbers, p and q, and compute the number n = pq.
- *n will be the modulus which will be used in both the public and private key.* 
- *The **key length** is the length of n in bits (for example 2048 bits, which is a common recommanded key length).* 
- *It is important to keep p and q secret since computing the value of n will allow anyone to "break" your encrypted messages.*
2) Then, compute the value _φ_(n) = (p-1)(q-1)
- *_φ_(n) is known as the Euler's totient function that counts the positive integers up to a given integer n that are relatively prime to n.*
- *The RSA algorithm correctness can be proved using Euler's theorem (see below)*
3) Choose a number e, such that 1 < e < _φ_(n) and e must be coprime with _φ_(n) (meaning gcd(e, _φ_(n)) = 1)
- *e is called the public exponent. It is part of the public key.*
- *The Public Key is the pair of values (n, e).*
4) Finally, determine d as d  ≡  $e^{-1}$  (mod  _φ_(_n_))
- *d is the private key (sometimes the Private Key is also written as the pair of values (n, d))*
- *d is the modular multiplicative inverse of e (modulo _φ_(n))*

Now we got our public and private keys ! We can now use them to encrypt or decrypt a message.

***Encrypt*** :
Assuming you want to communicate a message M, you must first ensure that M is an integer and that 0 ≤ M < n.
Then, you can compute the encrypted message C with :
$C \equiv M^{e} \pmod{n}$ 
(Note that the value of C will be  strictly less than n, since it's the modulus)

***Decrypt*** : 
To decrypt the ciphertext C, you now use the same kind of computation using the private key d : 
$M \equiv C^{d} \pmod{n}$ 


***Proof of correctness with Euler's Theorem :***

The RSA Algorithm "works" because we can proove using Euler's Theorem that any encrypted message M can be decrypted to recover the exact same M.

The Euler's theorem states that given any integer *n* > 0 and any integer *a* coprime with n we have :
$a^{(φ(n)) }$  ≡  1  (mod  n)

In RSA, the number *a* is actually our message M but since we built n to be a product of p and q, there are high chances that M is coprime with n. (If not, the algorithm works anyway but uses another proof with Fermat's little theorem)

Now, when we want to decrypt C (which is $M^{e}$), we do $C^{d}$ = $M^{ed}$.

Since we built *d* to be the modular multiplicative inverse of *e* (mod _φ_(n)) we have : ed ≡  1  (mod  _φ_(n)) <=> ed = 1 + kφ(_n_).

Then the proof is : $M^{ed}$ ≡ $M^{1 + kφ(n)}$ ≡ $M * M^{kφ(n)}$ ≡ $M*1^{k}$ ≡ $M$ (mod n)

We demonstrated that our private key *d* can be used to decrypt C and recover the original message M ! 

***Here is a quick implementation in Python of RSA, following the steps mentionned above :***

In [2]:
#Using math, we know that a number n is prime if we cannot find any divisor p of n with p  ≤ sqrt(n)
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(math.sqrt(num)) + 1):
        if num % i == 0:
            return False
    return True

#This function will generate random numbers until a prime number is generated
def generate_prime_number(low, high):
    while True:
        p = random.randint(low, high)
        if is_prime(p):
            return p

# Key Generation
def generate_keys():
    # Step 1 : Choose p and q, prime numbers, and compute n
    p = generate_prime_number(100, 300)  
    q = generate_prime_number(100, 300)
    while q == p:
        q = generate_prime_number(100, 300)
    
    n = p*q

    # Step 2 : Compute phi = (p-1)*(q-1)
    phi = (p-1)*(q-1)

    # Step 3 : Choose e that gcd(e, phi) = 1 and 1 < e < phi
    e = random.randint(2, (phi-1))
    while gcd(e, phi) != 1:
        e = random.randint(2, (phi-1))
    
    # Step 4 : Compute d, the modular multiplicative inverse of e (we can use python pow() function)
    d = pow(e, -1, phi) 

    public_key = (n, e)
    private_key = (n, d)
    return public_key, private_key

# Encrypting
def encrypt(m, public_key):
    n, e = public_key
    C = pow(m, e, n) #Using the definition of C above, we can use once again the pow() function to calculate the power with modulo
    return C

# Decrypting
def decrypt(c, private_key):
    n, d = private_key
    M = pow(c, d, n) 
    return M

public, private = generate_keys()
print("Public Key :", public)
print("Private Key :", private)

message = 12 #Test message for example (for text, we will have to convert it to integers first)

C = encrypt(message, public)
print("Encrypted message :", C)

M = decrypt(C, private)
print("Decrypted message :", M)

NameError: name 'gcd' is not defined

## II) Can we crack RSA ?

As you have seen before, factoring is very important in the field of cryptography, specifically in the RSA cryptosystem. 

The security of this system relies on the fact that our classical computers are not able to find (because it is hard and it is a computationally intensive task) the 2 factors of large composite numbers. 

Researchers have tried for many years to tackle this problem and, although they have developed increasingly sophisticated factoring algorithms (such as Pollard’s rho, the elliptic curve method, and the Number Field Sieve), these methods still require resources that grow so quickly with key size that properly chosen RSA moduli remain secure against practical classical attacks. 

In this part of the lab we will try to crack RSA by factoring large numbers and explore the practical limits of classical factoring techniques.

Firstly, let's try to brute force RSA

### Brute Force
---
Let $n = p q$ be the RSA modulus (product of two prime numbers $p$ and $q$) and let $L$ be the bit-length of $n$.  
There are two main brute-force approaches that could, in theory, recover the private key:

1. **Factorizing $n$**: recover $p$ and $q$ from $n$. Once $p$ and $q$ are known, compute  
   $$\varphi(n) = (p-1)(q-1)$$  
   and then the inverse $d$ of the public exponent $e$ modulo $\varphi(n)$, such that $d e \equiv 1 \pmod{\varphi(n)}$.
2. **Directly attacking $d$**: search for $d$ without explicitly factoring $n$ (for example, by exhaustively testing possible candidates for $d$ and checking their validity).

---

#### *Algorithm 1 — Exhaustive search of the private exponent $d$*
**Description.** Iterate over all possible values of $d$, and for each candidate, verify whether it is the modular inverse of $e$ (if $\varphi(n)$ is known), or attempt a test decryption and check the result.  
**Search space.** $1 \le d \le \varphi(n)$.  
**Complexity.** The number of trials is $\Theta(\varphi(n)) \approx \Theta(n)$. In terms of bit-length $L$:  
$$\Theta(2^{L})$$  
**Remark.** This is the most expensive brute-force approach and becomes infeasible as soon as $L$ is even moderately large.

---

#### *Algorithm 2 — Naïve factor search (test all integers $1,\dots,n$)*
**Description.** Test each integer $k \in \{1,\dots,n\}$ (or more efficiently, each prime number) to check if it divides $n$.  
**Complexity.** $\Theta(n) = \Theta(2^{L})$.  
**Remark.** The search stops as soon as a non-trivial factor $p$ is found, but the order of magnitude remains astronomically high.

---

#### *Algorithm 3 — Trial division up to $\sqrt{n}$*
**Description.** Test all integers (or preferably all prime numbers) $k$ such that $2 \le k \le \lfloor \sqrt{n} \rfloor$. As soon as one divisor is found, we obtain the factorization $n = k \cdot (n/k)$.  
**Justification.** If $n$ is composite, it necessarily has at least one non-trivial factor less than or equal to $\sqrt{n}$.  
**Complexity.** The number of trials is $\Theta(\sqrt{n})$. In terms of bit-length $L$:  
$$\Theta(2^{L/2})$$  
**Remark.** This is the best asymptotic complexity achievable under a purely exhaustive brute-force approach (without using advanced algebraic structure). However, it still grows exponentially in $L$.

---

### General Observations

Even with the improvement of restricting the search space to $\sqrt{n}$, the effort remains exponential in the bit-length $L$: the best brute-force complexity achievable is $\Theta(2^{L/2})$.  
Therefore, these methods are not feasible in real time for modern key sizes (e.g., $L \ge 2048$).

---

### Conclusion

- There are essentially two brute-force families: (1) factorizing $n$ to recover $p,q$ and then $d$, and (2) directly searching for $d$.  
- Naïve methods yield $\Theta(2^{L})$ complexity. The best brute-force approach (trial division up to $\sqrt{n}$) achieves $\Theta(2^{L/2})$.  
- All these methods remain exponential in $L$ and are therefore impractical for standard RSA key sizes.


---
### Implementation of brute force algorithm

In [3]:
n = 221
end_time = 0
start_time = time.time()
for i in range(2, n//2):
    if n%i == 0:
        end_time = time.time()
        print(f"Found the 2 factors : {i}, {n//i} in {end_time - start_time} seconds")
        break
if not end_time : 
    print("n is prime")

Found the 2 factors : 13, 17 in 0.00011181831359863281 seconds


#### Brute-Force RSA Cracking Timeline

| RSA key size (bits) | Total possibilities | Estimated time (1 GHz CPU) |
|-------------------|------------------|---------------------------|
| 128               | ~3.4×10³⁸        | ~10¹² years              |
| 256               | ~1.2×10⁷⁷        | ~10⁵¹ years              |
| 512               | ~1.3×10¹⁵⁴       | ~10¹²⁸ years             |
| 1024              | ~1.8×10³⁰⁸       | ~10²⁸² years             |
| 2048              | ~3.2×10⁶¹⁵       | ~10⁵⁸⁹ years             |
| 4096              | ~1.3×10¹²³⁰      | ~10¹²⁰⁴ years            |


### The GNFS Algorithm

#### Story of the GNFS

The General Number Field Sieve (GNFS) is the most efficient classical algorithm known for factoring large integers larger than $10^{100}$. Its history traces the evolution of computational number theory and reflects decades of research aimed at understanding the limits of public-key cryptography, particularly RSA.

The origins of the method go back to the 1980s, when John Pollard introduced the Number Field Sieve for a very special class of integers known as Cunningham numbers. Building on this idea, Arjen Lenstra, Hendrik Lenstra, Mark Manasse, and John Pollard generalized the approach in the early 1990s, creating the algorithm now known as the GNFS. This breakthrough provided, for the first time, a sub-exponential algorithm that could factor arbitrary large integers more efficiently than the Quadratic Sieve, which had been the previous state of the art.

From that point forward, GNFS became the central tool in record-setting factorizations. In 1994, it was used to factor RSA-129, a 129-digit number (426 bits) made famous by Martin Gardner’s challenge in Scientific American. Over the following decades, GNFS was refined and scaled up through distributed computing efforts and advances in algorithm engineering. Notable milestones include the factorization of RSA-768 (232 digits) in 2009, RSA-240 (795 bits) in 2019, and RSA-250 in 2020. Each of these achievements pushed the limits of available hardware and demonstrated both the power of GNFS and the ongoing need to increase RSA key sizes.

Today, GNFS remains the gold standard for classical integer factorization. Its history underscores the constant interplay between mathematical innovation and computational progress, and it provides the foundation for our understanding of how secure RSA remains in the face of classical attacks.

#### The algorithm

**I hope you are ready and you have fastened your seatbelts, because it isn't an easy algorithm to understand!**
We will be working through a specific numerical example to illustrate each concept along the way.

---

##### Step 0: Preliminary Idea – Difference of Squares

**Principle:** If we can find integers $s$ and $r$ such that:
$s^2 \equiv r^2 \pmod{n}$
then:
$n$ divides $(s - r)(s + r)$

**Factor extraction:** Let $n = p \cdot q$ (product of two primes). Then at least one of the following is a nontrivial factor of $n$ with high probability ($\sim 2/3$):
$\gcd(s + r, n)$ or $\gcd(s - r, n)$

*gcd meaning the greatest common divisor*

**Core idea of GNFS:** Reduce factoring $n$ to finding a congruence of squares modulo $n$:
$s^2 \equiv r^2 \pmod{n} \Rightarrow$ potential factorization via gcd.

***

##### Step 1: Choose $m$ and $f(x)$

We start by picking an integer $m$ close to $n^{1/d}$, where $d$ is the degree of the polynomial we want to construct. Using the base-$m$ expansion of $n$:
$$n = a_d m^d + a_{d-1} m^{d-1} + \dots + a_0$$we define the polynomial$$f(x) = a_d x^d + a_{d-1} x^{d-1} + \dots + a_0$$
so that $f(m) \equiv 0 \pmod{n}$. This polynomial encodes $n$ in a way that allows us to later find algebraic numbers whose squares are congruent modulo $n$, which is key to factorization.

**Implementation detail:** The base-$m$ expansion provides an easy way to construct a polynomial $f(x)$ with integer coefficients where $f(m)=n$. This ensures the crucial congruence $f(m) \equiv 0 \pmod n$ holds, which is a fundamental requirement for the mapping $\phi$ used later in the algorithm. For example, to factor $n=45113$, we can choose $m=31$ and use its base-31 expansion: $45113 = 1 \cdot 31^3 + 15 \cdot 31^2 + 29 \cdot 31 + 8$. From this, we can define the polynomial $f(x) = x^3 + 15x^2 + 29x + 8$.

***

##### Step 2: Define the rational factor base $R$

Next, we select a finite set of small prime numbers $R = \{p_1, p_2, \dots, p_k\}$. This **rational factor base** will be used to check if certain integers are “smooth,” meaning they factor completely over these primes. Working with smooth numbers ensures that we can later represent them as vectors for linear algebra over $\mathbb{F}_2$. $\mathbb{F}_2$ is a mathematical term that refers to the finite field with two elements. In simpler terms, it's a number system that only contains the numbers 0 and 1.

**Implementation detail:** The rational factor base is a collection of prime numbers. For practical purposes, it is often a set of small, consecutive primes up to a certain bound $M$, such as $R = \{p: \text{p is prime and } p \le M\}$. For the example of factoring $n=45113$, the rational factor base was chosen as all primes less than 30: $R = \{2, 3, 5, 7, 11, 13, 17, 19, 23, 29\}$. An integer $l$ is considered "smooth" over $R$ if all of its prime divisors are in $R$. This is a core concept that allows us to convert the factorization problem into a linear algebra problem over $\mathbb{F}_2$.

***

##### Step 3: Define the algebraic factor base $A$

In the first step, we built a list of small prime numbers to act as our "factoring toolkit" for regular integers. In this step, we're doing the same thing, but for a new, abstract number system.

Imagine we're not just working with numbers anymore, but with a new kind of "number plant" that has sprouted from our polynomial. These "number plants" are called **algebraic numbers**, and they look like $a + b\theta$. We need a way to factor these new "plants," so we create an **algebraic factor base**, which is our toolkit of special "algebraic prime seeds" for this new number system.

We identify these "prime seeds" as pairs of regular numbers, $(r_i, p_i)$, that satisfy a specific condition: when you plug $r_i$ into our original polynomial, the result is divisible by the prime $p_i$ (i.e., $f(r_i) \equiv 0 \pmod{p_i}$). This clever trick lets us use something we know (our polynomial) to find and categorize these new "algebraic primes."

**Implementation detail:** The goal is to find "number plants" ($a + b\theta$) that can be completely factored using only the "prime seeds" in our toolkit. We call these numbers "smooth," just as we did with the regular integers. This ensures that the algebraic side of our calculations is manageable and mirrors the integer side, providing two parallel sets of data that will eventually be combined. For example, with $n=45113$, we found our "prime seeds" by looking at all primes less than 90 : 
$A = \{ (0, 2), (6, 7), (13, 17), (11, 23), ... , (50, 73), (23, 79), (47, 79), (73, 79), (28, 89), (62, 89), (73, 89) \}$

***

##### Step 4: Define the quadratic character base $Q$

To capture additional congruence information, we choose a set of primes $Q$ not in $A$ and find integers $s$ such that $f(s) \equiv 0 \pmod{q}$. This **quadratic character base** helps control certain signs and parity conditions when combining smooth numbers, which is crucial for guaranteeing that the final products will form perfect squares.

**Implementation detail:** This base helps verify that an element in $\mathbb{Z}[\theta]$ is a perfect square. Each entry in $Q$ is a pair $(s, q)$, where $q$ is a prime not in $A$ and $s$ satisfies $f(s) \equiv 0 \pmod q$. These pairs are used to compute the Legendre symbol $\left(\frac{a+bs}{q}\right)$, which acts as a check for squareness. The number of entries in $Q$ can be increased to raise the probability that the final result is a perfect square. For $n=45113$, primes 97, 101, 103, and 107 were used to build $Q$. $Q = \{(28, 97), (87, 101), (47, 103), (4, 107), (8, 107), (80, 107)\}$

***

##### Step 5: Find smooth pairs $(a, b)$

We then search for integer pairs $(a, b)$ such that both $a + bm$ and $a + b\theta$ are smooth over the rational and algebraic factor bases, respectively. By looping over $a$ for fixed $b$ and checking factorability, we collect enough pairs (typically more than $1 + k + l + u$) to ensure the linear system in the next step will have nontrivial solutions. These **smooth pairs** form the building blocks for constructing a congruence of squares.

*1+k+l+u represents the total number of columns in the matrix we are building, where 1 is for the sign bit, k is the number of primes in the rational factor base, l is the number of "algebraic primes" in the algebraic factor base, and u is the number of primes in the quadratic character base.*

**Implementation detail:** This is the core "sieving" part of the algorithm. We fix a small integer $b$ and let $a$ vary over a range, for example from $-N$ to $N$. For each pair $(a, b)$, we test if $a+bm$ is smooth over the rational factor base $R$ and if $a+b\theta$ is smooth over the algebraic factor base $A$. This is done using sieve arrays. An integer $q \in R$ divides $a+bm$ if $a \equiv -bm \pmod q$, while an algebraic factor $(r_i, p_i) \in A$ divides $a+b\theta$ if $a \equiv -br_i \pmod{p_i}$. We continue this process with different values of $b$ until we have found a sufficient number of smooth pairs, which is typically more than the total number of elements in all three bases ($1+k+l+u$).

***

##### Step 6: Construct the matrix $X$

Each smooth pair $(a, b)$ is represented as a row vector containing the sign of $a + bm$, the exponents of its rational prime factors modulo 2, the exponents of the algebraic factors modulo 2, and any relevant quadratic characters. Stacking these vectors forms a $y \times (1+k+l+u)$ matrix $X$. This matrix encodes all information necessary to combine smooth numbers into perfect squares.

**Implementation detail:** For each smooth pair $(a,b)$, a row vector is created. The first entry is a sign bit (0 for positive $a+bm$, 1 for negative). The next $k$ entries are the exponents of the primes in the rational factor base $R$ from the factorization of $a+bm$, taken modulo 2. The following $l$ entries are the exponents of the algebraic factors from the factorization of $a+b\theta$, also modulo 2. Finally, the last $u$ entries are derived from the quadratic character base $Q$, representing whether the Legendre symbol $\left(\frac{a+bs}{q}\right)$ is 1 (entry 0) or -1 (entry 1). This vector essentially summarizes the parity of the exponents and the quadratic characters, which is a concise way to check for perfect squares.

***

##### Step 7: Solve the linear system modulo 2

We solve the system
$$X^T \cdot \mathbf{A} \equiv 0 \pmod{2}$$to find subsets of smooth pairs that, when multiplied together, produce a perfect square both in $\mathbb{Z}$ and in $\mathbb{Z}[\theta]$. Let $V \subset U$ denote the selected subset. By construction:
$$\prod_{(a_j, b_j) \in V} (a_j + bm) \text{ is a perfect square in } \mathbb{Z},$$
$$\prod_{(a_j, b_j) \in V} (a_j + b_j\theta) \text{ is a perfect square in } \mathbb{Z}[\theta]$$
This gives us the congruence of squares needed to attempt factorization.

**Implementation detail:** The matrix equation $X^T \mathbf{A} \equiv \mathbf{0} \pmod 2$ is a system of linear equations over the field $\mathbb{F}_2$. The entries of the vector $\mathbf{A}$ are either 0 or 1, and a non-trivial solution is guaranteed to exist if the number of rows is greater than the number of columns ($y > 1+k+l+u$). Each $A_j=1$ in the solution vector corresponds to including the $j$-th smooth pair $(a_j, b_j)$ in the product. When the vectors corresponding to these pairs are added modulo 2, the result is the zero vector, which means that all exponents (and signs) in the final product are even, a necessary condition for being a perfect square.

***

##### Step 8: Factor $n$ using the difference of squares

Finally, we map the algebraic product back to integers using
$$x^2 = \phi \left( \prod_{(a_j, b_j) \in V} (a_j + b_j\theta) \right),$$$$y^2 = \prod_{(a_j, b_j) \in V} (a_j + bm)$$and try to extract factors via$$\gcd(n, x+y) \quad \text{and} \quad \gcd(n, x-y)$$
If a nontrivial factor is found, the algorithm succeeds. Otherwise, we repeat the process with a new factor base or additional smooth pairs. This step exploits the classical “difference of squares” idea: if $s^2 \equiv r^2 \pmod{n}$, then $\gcd(s \pm r, n)$ often reveals a nontrivial factor of $n$.

**Implementation detail:** With the subset of pairs $V$ from the previous step, we compute the products to get a perfect square in $\mathbb{Z}$ and a perfect square in $\mathbb{Z}[\theta]$. We then compute the integer value of the algebraic perfect square using the mapping $\phi: \mathbb{Z}[\theta] \rightarrow \mathbb{Z}/n\mathbb{Z}$. This mapping satisfies $\phi(\theta) \equiv m \pmod n$. Thus, if $\beta^2 = \prod_{(a_j, b_j) \in V} (a_j + b_j\theta)$, then $\phi(\beta^2) = \phi(\beta)\phi(\beta)$ and $\phi(\beta^2) = \phi(\prod_{(a_j, b_j) \in V} (a_j + b_j\theta)) = \prod_{(a_j, b_j) \in V} (a_j + b_jm)$. This creates the congruence of squares $x^2 \equiv y^2 \pmod n$ that can be used for factorization with a high probability of success. For example, in the case of $n=45113$, this process led to the discovery of factors 197 and 229.

#### Complexity

Heuristically, its complexity for factoring an integer `n` (consisting of $\lfloor\log_2 n\rfloor + 1$ bits) is of the form:

![alt text](GNFS_complexity.svg)

#### Code in python

**Disclaimer:**  
This Python implementation of the General Number Field Sieve (GNFS) is designed for **educational purposes only**.  
It is **not optimized for performance** and will run **very** slowly.  
In practice, highly optimized GNFS implementations are written in **C or C++** with advanced sieving, memory management, and parallelization to handle real-world cryptographic-sized numbers efficiently.

In [4]:
# ------------------------
# Step 1: Base-m polynomial selection
# ------------------------
def base_m_expansion(n, m):
    """Expand n in base m up to the given degree."""
    coeffs = []
    temp = n
    degree = 0
    while temp // m**degree != 0:
        degree += 1
    for _ in range(degree + 1):
        coeffs.append(temp % m)
        temp //= m
    return coeffs[::-1]  # highest-degree first

def construct_polynomial_from_m(n, m):
    """Construct polynomial f(x) such that f(m) = n."""
    coeffs = base_m_expansion(n, m)
    return coeffs

def evaluate_polynomial(coeffs, x):
    """Evaluate polynomial at x."""
    result = 0
    deg = len(coeffs) - 1
    for i, c in enumerate(coeffs):
        result += c * (x ** (deg - i))
    return result

def polynomial_function(coeffs):
    """Return a lambda function f(x) from coefficients."""
    return lambda x: evaluate_polynomial(coeffs, x)

In [5]:
# ------------------------
# Step 2: Rational factor base R
# ------------------------
def rational_factor_base(bound):
    return [p for p in range(2, bound+1) if isprime(p)]

def factorize_over_base(x, base):
    exponents = []
    for p in base:
        count = 0
        while x % p == 0:
            x //= p
            count += 1
        exponents.append(count)
    return exponents, x  # remaining cofactor

In [6]:
# ------------------------
# Step 3: Algebraic factor base A
# ------------------------
def algebraic_factor_base(f, primes):
    A = []
    for p in primes:
        for r in range(p):
            if f(r) % p == 0:
                A.append((r, p))
    return A

In [7]:
# ------------------------
# Step 4: Quadratic character base Q
# ------------------------
def quadratic_character_base(f, primes, exclude_A):
    Q = []
    for q in primes:
        if q in exclude_A:
            continue
        for s in range(q):
            if f(s) % q == 0:
                Q.append((s, q))
    return Q

In [8]:
# ------------------------
# Step 5: Find smooth pairs (a, b)
# ------------------------
def factorize_with_sieve(x, factor_base_primes, spf):
    if x <= 1:
        return [0] * len(factor_base_primes), 1
        
    exponents = [0] * len(factor_base_primes)
    temp_x = x
        
    while temp_x > 1:
        p = spf[temp_x]
        try:
            idx = factor_base_primes.index(p)
            exponents[idx] += 1
        except ValueError:
            return [0] * len(factor_base_primes), temp_x  # Not smooth
        temp_x //= p
    return exponents, 1
    
def find_smooth_pairs(n, m, f, R, A, Q, a_limit=50, b_limit=3):
    pairs = []
    A_primes = [p for _, p in A]
    
    # Pre-compute primes for sieve
    all_primes = sorted(list(set(R + A_primes)))
    max_val_rational = b_limit * m + a_limit
    max_val_algebraic = b_limit * f(m) + a_limit
    max_val = max(max_val_rational, max_val_algebraic)
    
    # Sieve to find smallest prime factor for all numbers up to max_val
    spf = list(range(max_val + 1))
    for i in range(2, int(max_val**0.5) + 1):
        if spf[i] == i:  # i is prime
            for j in range(i * i, max_val + 1, i):
                if spf[j] == j:
                    spf[j] = i

    R_dict = {p: i for i, p in enumerate(R)}
    A_dict = {p: i for i, p in enumerate(A_primes)}
    
    for b in range(1, b_limit):
        for a in range(-a_limit, a_limit):
            val_r = a + b * m
            val_a = a + b * f(m)
            
            rat_exp, rem_r = factorize_with_sieve(abs(val_r), R, spf)
            if rem_r != 1:
                continue

            alg_exp, rem_a = factorize_with_sieve(abs(val_a), A_primes, spf)
            if rem_a != 1:
                continue
            
            quad_exp = [0 if legendre_symbol(a + b * s, q) == 1 else 1 for s, q in Q]
            
            pairs.append({
                "a": a, "b": b,
                "rational": rat_exp,
                "algebraic": alg_exp,
                "quadratic": quad_exp,
                "sign_r": 0 if val_r >= 0 else 1,
                "sign_a": 0 if val_a >= 0 else 1
            })
            
    return pairs

In [9]:
# ------------------------
# Step 6: Construct matrix X
# ------------------------
def construct_matrix_X(pairs, R_len, A_len, Q_len):
    matrix = []
    for p in pairs:
        row = [p["sign_r"], p["sign_a"]] + p["rational"] + p["algebraic"] + p["quadratic"]
        matrix.append([x % 2 for x in row])
    return np.array(matrix, dtype=int)

In [10]:
# ------------------------
# Step 7: Solve linear system modulo 2
# ------------------------
def solve_linear_system_mod2(X):
    mat = X.copy()
    rows, cols = mat.shape
    pivot_rows = []
    dependencies = []

    for col in range(cols):
        pivot_row = None
        for r in range(rows):
            if r in pivot_rows:
                continue
            if mat[r, col] == 1:
                pivot_row = r
                pivot_rows.append(r)
                break
        if pivot_row is None:
            continue
        for r in range(rows):
            if r != pivot_row and mat[r, col] == 1:
                mat[r] = (mat[r] + mat[pivot_row]) % 2

    for i, row in enumerate(mat):
        if not row.any():
            dependencies.append(i)
    return dependencies if dependencies else [0]

In [11]:
# ------------------------
# Step 8: Factor n using difference of squares
# ------------------------
def difference_of_squares_factor(n, s, r):
    factor1 = gcd(s + r, n)
    factor2 = gcd(s - r, n)
    factors = [f for f in (factor1, factor2) if 1 < f < n]
    return factors[0] if factors else None

def factor_n_from_pairs(n, pairs, dependency, m, f):
    x_prod = 1
    y_prod = 1
    for idx in dependency:
        a = pairs[idx]["a"]
        b = pairs[idx]["b"]
        x_prod *= (a + b * m)
        y_prod *= (a + b * f(m))
    
    x = abs(x_prod)
    y = int(math.isqrt(abs(y_prod)))
    
    factor = difference_of_squares_factor(n, x, y)
    return factor

In [12]:
def polynomial_to_string(coeffs):
    """Convert a list of coefficients into a polynomial string. Only for print purposes"""
    degree = len(coeffs) - 1
    terms = []
    for i, c in enumerate(coeffs):
        power = degree - i
        if c == 0:
            continue
        if power == 0:
            terms.append(f"{c}")
        elif power == 1:
            terms.append(f"{c}*x")
        else:
            terms.append(f"{c}*x**{power}")
    return " + ".join(terms)

Here is all the functions you needed to construct the GNFS algorithm. Try to implement the full algorithm yourself.

In [13]:
def gnfs(n, m, R_bound=30, A_bound=90, Q_bound=107, a_limit=20, b_limit=3):
    coeffs = construct_polynomial_from_m(n, m)
    f = polynomial_function(coeffs)
    print(f"Polynomial f(x) = {polynomial_to_string(coeffs)}")

    R = rational_factor_base(R_bound)
    print(f"Rational factor base R: {R}")
    
    A_primes = [p for p in range(2, A_bound) if isprime(p)]
    A = algebraic_factor_base(f, A_primes)
    print(f"Algebraic factor base A: {A}")
    
    Q_primes = [p for p in range(A_bound + 1, Q_bound + 1) if isprime(p)]
    Q = quadratic_character_base(f, Q_primes, [p for _, p in A])
    print(f"Quadratic character base Q: {Q}")
    
    pairs = find_smooth_pairs(n, m, f, R, A, Q, a_limit=a_limit, b_limit=b_limit)
    print(f"Found {len(pairs)} smooth pairs")
    
    num_cols = len(R) + len(A) + len(Q) + 2
    if len(pairs) < num_cols:
        print("Not enough smooth pairs. Increase limits (a and b) or bounds (R, A, Q).")
        return None

    X = construct_matrix_X(pairs, len(R), len(A), len(Q))
    dependency = solve_linear_system_mod2(X)

    if not dependency:
      print("No non-trivial dependency found.")
      return None

    factor = factor_n_from_pairs(n, pairs, dependency, m, f)
    return factor

In [14]:
def gnfs_no_print(n, m, R_bound=30, A_bound=90, Q_bound=107, a_limit=20, b_limit=3):
    coeffs = construct_polynomial_from_m(n, m)
    f = polynomial_function(coeffs)
    R = rational_factor_base(R_bound)
    A_primes = [p for p in range(2, A_bound) if isprime(p)]
    A = algebraic_factor_base(f, A_primes)
    Q_primes = [p for p in range(A_bound + 1, Q_bound + 1) if isprime(p)]
    Q = quadratic_character_base(f, Q_primes, [p for _, p in A])
    pairs = find_smooth_pairs(n, m, f, R, A, Q, a_limit=a_limit, b_limit=b_limit)
    num_cols = len(R) + len(A) + len(Q) + 2
    if len(pairs) < num_cols:
        return None
    X = construct_matrix_X(pairs, len(R), len(A), len(Q))
    dependency = solve_linear_system_mod2(X)
    if not dependency:
      return None
    factor = factor_n_from_pairs(n, pairs, dependency, m, f)
    return factor

Let's run it on an example. (This example will work but if you want to try different values feel free but it might not work)

In [15]:
n = 45113
m = 31
print(f"Trying to factor {n} with m={m}...")
start_time = time.time()
factor = gnfs(n, m, R_bound=75, A_bound=150, Q_bound=167, a_limit=770, b_limit=753)
if factor:
    end_time = time.time()
    print(f"Found the 2 factors : {factor} and {n // factor} in {end_time - start_time} seconds")
    print(f"{n} = {factor} * {n // factor}")
else:
    print(f"Could not factor {n}")
    print(f"Try increasing limits (a and b) or bounds (R, A, Q) or use different values of m")

Trying to factor 45113 with m=31...
Polynomial f(x) = 1*x**3 + 15*x**2 + 29*x + 8
Rational factor base R: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73]
Algebraic factor base A: [(0, 2), (6, 7), (13, 17), (11, 23), (26, 29), (18, 31), (19, 41), (13, 43), (1, 53), (46, 61), (2, 67), (6, 67), (44, 67), (50, 73), (23, 79), (47, 79), (73, 79), (28, 89), (62, 89), (73, 89), (28, 97), (87, 101), (47, 103), (4, 107), (8, 107), (80, 107), (52, 109), (99, 109), (108, 113), (33, 139)]
Quadratic character base Q: [(11, 151), (42, 151), (83, 151), (67, 157), (56, 163), (137, 167)]
Found 556 smooth pairs
Found the 2 factors : 197 and 229 in 11.219705820083618 seconds
45113 = 197 * 229


Here is a code to try to find good values for the bounds and limits :

In [ ]:
n = 229*991
m = 5
factor = False
i = 0
while not factor :
    start_time = time.time()
    factor = gnfs_no_print(n, m, R_bound=i, A_bound=2*i, Q_bound=2*i+17, a_limit=20+10*i, b_limit=3+10*i)
    if factor:
        end_time = time.time()
        print(f"Realised factorisation with i = {i}")
        print(f"Found the 2 factors : {factor} and {n // factor} in {end_time - start_time} seconds")
        print(f"{n} = {factor} * {n // factor}")
    else:
        i += 1

#### GNFS Cracking Time Estimates (Classical Resources)

For reference, the following table estimates show how long it would take to factor RSA keys of different sizes. The times are extrapolated from data on previously factored, smaller numbers.

| RSA key size (bits) | Approx. classical cost (core-years, GNFS-style) | 1 core | 100 cores | 1 k cores | 10 k cores | 1 M cores |
|---------------------|-----------------------------------------------:|--------------:|----------:|----------:|-----------:|-----------:|
| **512**             | ~10 core-years                                 | ~10 years     | ~36 days  | ~3.6 days | ~8.6 hours | ~5 minutes | 
| **768**             | ~600 core-years                                | ~600 years    | ~6 years  | ~2.2 years| ~7.9 months| ~5 days    | 
| **1024**            | ~30,000 core-years                             | ~30,000 years | ~300 years| ~30 years | ~3 years   | ~10 days   | 
| **2048**            | ~1×10⁹ core-years                              | ~1B years     | ~10M years| ~1M years | ~100k years| ~1,000 years| 
| **4096**            | ~10¹⁵ core-years                               | ~10¹⁵ years   | ~10¹³ years| ~10¹² years| ~10¹¹ years| ~10⁹ years |



As we can see, classical computers have no hope of cracking RSA today. But what about the emerging world of quantum computing ? Could it succeed where classical machines fail?

## III) Discover Quantum Computers and Qiskit

### Setup

### First Algorithm (QRNG)

## IV) Crack RSA with Quantum Computers

### History of the Shor Algorithm

### Implement the Shor Algorithm

### Test the Shor algorithm on RSA

## V) Is security of our data compromised ?

### Quantum Computers are yet not powerful enough

### Classical methods Discrete Log Problem (Elliptic Curve) cannot be crack by quantum computers (yet)

### Quantum Cryptography based on entanglement